# Визуализация данных временного ряда

## Введение

Данные, предтсавленные в одной точке времени, известны как *`кросс-секционные данные`*. Как специалист по данным или аналитик, вы иногда можете столкнуться с данными, которые собираются в течение определенных периодов времени, известными как *`данные временных рядов`*.

Данные временных рядов появляются в реальном мире довольно часто. Например, показания погоды, цены на акции компании и данные о продажах - все это примеры данных, которые можно отслеживать с течением времени. Поэтому важно, чтобы вы могли исследовать и визуализировать данные с компонентом времени.

В этой статье вы узнаете, как исследовать данные временных рядов с помощью Python, используя следующее
- линейные графики
- диаграммы ящиков
- тепловые карты
- графики задержек
- графики автокорреляции

## Линейный график

Линейный график обычно используется для визуализации данных временного ряда.

На линейном графике время обычно находится на оси Х, а значения наблюдения - на оси Y. Давайте покажем пример этого графика, используя CSV-файл с данными о продажах для малого бизнеса за пятилетний период.

Сначала давайте импортируем несколько полезных библиотек Python и загрузим наши данные:

```python
# import libraries
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

# load in data
sales_data = pd.read_csv("sales_data.csv")

# peek at first few rows of data
sales_data.head()
```

Вот первые несколько строк данных о продажах

```text
| дата | продажи
— | — | — 0 | 01.01.2016 | 2000.0 1 | 2016-01-02 | 1700,0 2 | 03.01.2016 | 1800,0 3 | 04.01.2016 | 1400,0 4 | 05.01.2016 | 1500,0
```

Давайте создадим линейный график данных, отложив дату на оси X, а продажи - на оси Y.

```python
# convert string to datetime64
sales_data["date"] = sales_data["date"].apply(pd.to_datetime)
sales_data.set_index("date", inplace=True)

# create line plot of sales data
plt.plot(sales_data["date"], sales_data["sales"])
plt.xlabel("Date")
plt.ylabel("Sales (USD)")
plt.show()
```

![](./img/3_2_1.png)

Обратите внимание, как мы можем видеть тенденция данных с течением времени. Глядя на график, кажется, что
- Продажи носят сезонный характер, достигая пика в начале и конце каждого года и замедляясь в середине каждого года.
- Продажи, похоже, не показывают признаков роста с течением времени. Похоже, это застойный бизнес.

## Диаграмма ящика

При работе с данными временных рядов диаграмма ящиков могут быть полезны для просмотра распределения значений, сгруппированных по временному интервалу.

Например, давайте создадим диаграмму продаж для каждого года и расположим их рядом для сравнения

```python
# extract year from date column
sales_data["year"] = sales_data["date"].dt.year

# box plot grouped by year
sns.boxplot(data=sales_data, x="year", y="sales")
plt.show()
```

![](./img/3_2_2.png)

Для каждого года данных о проадаж мы можем легко увидеть полезную информацию, такую как медианный объем продаж, максимальный и минимальный объем продаж, межквартильный размех наших данных и любые выборсы.

Медианные показатели продаж за каждый год (представленные горизонтальной линией в каждом блоке) довольно стабильны, что говорит о том, что продажи не растут с течением времени.

## Тепловая карта

Мы также можем использовать тепловую карту для сравнения наблюдений между временными интервалами в данных временных рядов.

Например, давайте создадим тепловую карту плотности с годом на Y и месяцем на оси X. Это можно сделать, вызвав *`heatmap()`* функцию *`sns`* объекта Seaborn

```python
# calculate total sales for each month
sales = sales_data.groupby(["year", "month"]).sum()

# re-format the data for the heat-map
sales_month_year = sales.reset_index().pivot(index="year", columns="month", values="sales")

# create heatmap
sns.heatmap(sales_month_year, cbar_kws={"label": "Total Sales"})
plt.title("Sales Over Time")
plt.xlabel("Month")
plt.ylabel("Year")
plt.show()
```

![](./img/3_2_3.png)

Напомним, что на тепловой карте по мере того, как цвет становится ярче и переходит от темно-фиолетового к желтому, общий объем продаж в соответствующей ячейке увеличивается.

Здесь мы снова видим, что продажи довольно стабильны из года в год и также демонстрируют сезонность.

## Диаграмма разброса задержек

Мы можем использовать *`диаграмму рассеяния задержек`*, чтобы исследовать связь между наблюдением и задержклй этого наблюдения.

Во временном ряду задержка - это предыдущее наблюдение
- Наблюдение на предыдущем временном шаге (наименьший временной интервал, для которого у нас есть отдельные измерения) называется задержкой 1.
- Наблюдение, произошедшее два раза назад, называется задержкой 2 и т.д.

В наборе данных о продажах у нас есть разные значения продаж для каждого дня. Поэтому значение лага 1 для любого конкретного дня равно продажам в предыдущий день. Значение лага 2 - это продажи два дня назад и т.д.

Модуль *`plotting`* библиотеки *`pandas`* имеет встроенную *`lag_plot`* функцию, которая отображает наблюдение с задержкой *`1(t+1)`* на оси Y

```python
# import lag_plot function
from pandas.plotting import lag_plot

# lag scatter plot
lag_plot(sales_data)
plt.show()
```

![](./img/3_2_4.png)

Как можно интерпретировать диаграмму рассеяния с задержкой?
- Если точки перемещаются из нижнего левого угла в верхний правый, это указывает на положительную корреляцию между наблюдениями и их значениями лага 1. Например, высокие продажи в один день связаны с высокими продажами в предыдущий день.
- Если точки перемещаются из верхнего левого угла в нижний правыйЮ жто указывает на отрицательную корреляцию между наблюдениями и их значениями лага 1. Например, высокие продажи в один день связаны с низкими продажами в предыдущий день и наоборот.
- Если на графике задержек нет идентифицируемой структуры, это указывает на то, что данные случайны, и нет никакой связи между значениями в последовательных временных точках. Например, продажи в один день не дают вам никакой информации об ожидаемых продажах на следующий день

Изучение связи между наблюдением и задержкой этого наблюдения полезно для определения того, является ли набор данных случайным.

Поскольку точки в данных о продажах движутся по диагональной линии из нижнего левого угла в верхний правый, это указывает на то, что наши данные не случайны и существует положительная корреляция между наблюдениями и их значениями запаздывания 1

## График автокорреляции

График автокорреляции используется для того, чтобы показать, являются ли элементы временного ряда положительно коррелированными, отрицательно коррелированными или независимыми друг от друга.

Это можно изобразить с помощью *`autocorrelation_plot()`* функции *`pandas.plotting`* модуля

```python
# import autocorrelation function
from pandas.plotting import autocorrelation_plot

# autocorrelation plot
autocorrelation_plot(sales_data)
plt.show()
```

![](./img/3_2_5.png)

На графике автокорреляции выше задержка находится на оси X, а значение автокорреляции, которое варьируется от -1 до 1, находится на оси Y. Значение, близкое к 0, указывает на слабую корреляцию, тогда как значения, близкие к -1 и 1, указывают на сильную корреляцию.

Обратите внимание, как график автокорреляции для данных о продажах формирует волны, колеблющиеся между сильной отрицательной и положительной корреляцией. Эти волны указывают на то, что нам набор данных демонстрирует сезонность.

Также обратите внимание, как автокорреляция уменьшается с течением времени. Это указывает на то, что продажи, как правило, схожи в последовательные дни, но продажи трехлетный давности меньше связаны с сегодняшними продажами, чем продажи годичной давности.

# Обзор

В этой статье вы получили краткое введение в изучение и визуализацию данных временных рядов с использованием
- Линейных графиков
- Диаграммы ящиков
- Тепловые карты
- Графики задержек
- Графики автокорреляции

Как специалист по данным или аналитик, вы часто будете работать с данными, которые меняются со временем. Двигаясь вперед, вы теперь лучше подготовлены к исследованию и визуализации данных с пременным компонентом.